# Build and Monitor a Web Research Agent with Exa, LangGraph & Quotient

<a target="_blank" href="https://colab.research.google.com/github/quotient-ai/quotient-alpha/blob/main/cookbooks/agents/research/exa-quotient-agent.ipynb">
 <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook shows how to build a LangGraph-based **research assistant** powered by [Exa](https://exa.ai) and Anthropic Claude. The agent answers real-world search queries using live web content via Exa's semantic search, and is monitored using [Quotient AI](https://www.quotientai.co/) to detect hallucinations, irrelevant retrievals, and other failure modes.


We'll use API keys from:
 - [Anthropic](https://www.anthropic.com/) — get your API key from the [Anthropic Console](https://console.anthropic.com/)
 - [Exa](https://exa.ai) — get your API key from the [Exa Dashboard](https://dashboard.exa.ai)
 - [Quotient AI](https://www.quotientai.co) — get your API key from the [Quotient AI app](https://app.quotientai.co)
 
Both Exa and Quotient offer generous free tiers to get started; you can check out their pricing [here](https://exa.ai/pricing) and [here](https://www.quotientai.co/pricing).


In [1]:
import os
# Set API keys:
os.environ['ANTHROPIC_API_KEY'] = "anthropic-api-key"
os.environ['EXA_API_KEY'] = "exa-api-key"
os.environ['QUOTIENT_API_KEY'] = "quotient-api-key"

In [2]:
%pip install -qU langchain-anthropic langchain-exa langgraph quotientai


Note: you may need to restart the kernel to use updated packages.


### Step 1: Set Up a Research Agent with Exa + Anthropic

In this step, we define a LangGraph agent that uses **ExaSearchRetriever** to gather live web content and Anthropic's Claude model for reasoning.

What's happening here:

- `ExaSearchRetriever` allows the agent to perform semantic web searches with highlights
- `ChatAnthropic` initializes Claude as the core reasoning engine
- We use LangGraph's `StateGraph` to create a workflow that:
  - Takes user queries
  - Searches the web with Exa
  - Processes results with Claude
  - Returns comprehensive answers


In [3]:
from typing import List, Literal
from datetime import datetime

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool
from langchain_exa import ExaSearchRetriever
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode


@tool
def retrieve_web_content(query: str) -> List[str]:
    """Function to retrieve usable documents for AI assistant"""
    # Initialize the Exa Search retriever
    retriever = ExaSearchRetriever(k=3, highlights=True)

    # Define how to extract relevant metadata from the search results
    document_prompt = PromptTemplate.from_template(
        """
    <source>
        <url>{url}</url>
        <highlights>{highlights}</highlights>
    </source>
    """
    )

    # Create a chain to process the retrieved documents
    document_chain = (
        RunnableLambda(
            lambda document: {
                "highlights": document.metadata.get("highlights", "No highlights"),
                "url": document.metadata["url"],
            }
        )
        | document_prompt
    )

    # Execute the retrieval and processing chain
    retrieval_chain = retriever | document_chain.map()

    # Retrieve and return the documents
    documents = retrieval_chain.invoke(query)
    return documents


# Determine whether to continue or end
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    return "tools" if last_message.tool_calls else END

# Get current date for temporal context
current_date = datetime.now().strftime("%B %d, %Y")

# Create system prompt with current date
system_prompt = SystemMessage(content=f"""You are a helpful assistant that answers questions based on provided context.
                            Answer the following question using ONLY the provided context. If the context doesn't contain enough information to fully answer the question, acknowledge what information is missing.
                            Today's date is {current_date}.
                            """)
    
def create_agent():
    """Create a fresh instance of the agent"""

    # Function to generate model responses
    def call_model(state: MessagesState):
        # Add system message if it's not already present
        messages = state["messages"]
        if not any(isinstance(msg, SystemMessage) for msg in messages):
            messages = [system_prompt] + messages
        response = model.invoke(messages)
        return {"messages": [response]}

    # Define and bind the AI model
    model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(
        [retrieve_web_content]
    )

    # Define the workflow graph
    workflow = StateGraph(MessagesState)
    workflow.add_node("agent", call_model)
    workflow.add_node("tools", ToolNode([retrieve_web_content]))
    workflow.set_entry_point("agent")
    workflow.add_conditional_edges("agent", should_continue)
    workflow.add_edge("tools", "agent")

    # Compile the workflow into a runnable (without memory)
    return workflow.compile()

## Step 2: Initialize the Quotient SDK to Monitor the Agent

In this step, we set up [Quotient AI](https://www.quotientai.co) to **monitor our LangGraph agent** as it answers real-world queries.

Quotient allows us to:

- Log each query and model response
- Attach the retrieved documents for grounding checks
- Automatically detect hallucinations (i.e., unsupported claims) and irrelevant documents

The configuration below tells Quotient to log 100% of interactions and run hallucination detection on every one. This will give us full visibility into how our agent is performing, and whether it's staying grounded in the context it retrieves.


In [4]:
from quotientai import QuotientAI

# Initialize the Quotient SDK
quotient = QuotientAI()

quotient.logger.init(
    # Name your application or project
    app_name="exa-agent",
    # Set the environment (e.g., "dev", "prod", "staging")
    environment="test",
    # Set the sample rate for logging (0-1)
    sample_rate=1.0,
    # Enable hallucination detection
    hallucination_detection=True,
    # Set the sample rate for detections (0-1)
    hallucination_detection_sample_rate=1.0,
)


## Step 3: Run Queries Through the Agent and Log Responses to Quotient

In this step, we simulate real user queries by reading from a `.jsonl` file and sending each query to the LangGraph agent. For each query, we:

1. **Invoke the agent** using the LangGraph app, which triggers Claude + Exa tool calls
2. **Capture the final model response** from the messages
3. **Extract all documents** returned by Exa's semantic search
4. **Format documents** into a structured list of `{"page_content": ..., "metadata": ...}` to support downstream evaluation
5. **Log the full interaction to Quotient**, including:
   - The original query
   - The model's answer
   - The retrieved documents for grounding checks
   - Metadata such as model version for traceability

Each interaction is logged with `quotient.log(...)`, enabling automatic hallucination detection and structured evaluation inside the Quotient platform.


In [5]:
import json
import random
import re
import ast
from typing import List

def parse_string_prompt_value(text: str) -> List[dict]:
    """Parse StringPromptValue responses into documents"""
    # First, find all StringPromptValue instances in the content
    pattern = r'StringPromptValue\(text="(.*?)"\)'  # Simplified pattern
    matches = re.findall(pattern, text, re.DOTALL)
    
    documents = []
    for match in matches:
        try:
            # Extract URL and highlights from the XML-like structure
            url_match = re.search(r'<url>(.*?)</url>', match)
            highlights_match = re.search(r'<highlights>\[(.*?)\]</highlights>', match)
            
            url = url_match.group(1) if url_match else "No URL"
            highlights_str = highlights_match.group(1) if highlights_match else ""
            
            # Clean up the highlights string and convert to list
            highlights = [h.strip().strip("'") for h in highlights_str.split("|") if h.strip()]
            
            # Format document for Quotient
            doc = {
                "page_content": match,  # The full content
                "metadata": {
                    "url": url,
                    "highlights": highlights
                }
            }
            documents.append(doc)
        except Exception as e:
            print(f"⚠️ Error parsing StringPromptValue: {str(e)}")
            continue
    
    return documents

# Load queries from file
with open("search_queries.jsonl") as f:
    all_queries = [json.loads(line)["question"] for line in f]

log_ids = []
num_queries = 10

# Randomly select queries
queries = random.sample(all_queries, min(num_queries, len(all_queries)))

# Run each query through the agent
for i, query in enumerate(queries[:num_queries]):
    print(f"\n🧠 Query: {query}")
    
    # Run the query through the LangGraph app
    print("\n🤖 Running query through agent...")

    app = create_agent()
    
    final_state = app.invoke(
        {"messages": [HumanMessage(content=query)]},
        config={"configurable": {"thread_id": i}},
    )
    
    # Get the final response
    final_response = final_state["messages"][-1].content
    print(f"\n➡️ Final answer: {final_response}")

    # Collect all documents from tool results
    formatted_docs = []

    messages = final_state["messages"]
    
    for j, msg in enumerate(messages):
        if isinstance(msg.content, str) and "StringPromptValue" in msg.content:
            # Direct StringPromptValue in message content
            docs = parse_string_prompt_value(msg.content)
            formatted_docs.extend(docs)

    # Log to Quotient
    if formatted_docs:
        try:
            log_id = quotient.log(
                user_query=query,
                model_output=final_response,
                documents=formatted_docs,
                tags={
                    'model': "claude-3-7-sonnet-20250219",
                    'thread_id': i
                }
            )
            print(f"📝 Logged to Quotient with log_id: {log_id}")
            log_ids.append(log_id)
        except Exception as e:
            print(f"❌ Error logging to Quotient: {str(e)}")
    else:
        print("⚠️ No documents were collected to log")

print(f"\n✅ Successfully logged {len(log_ids)} queries to Quotient")


🧠 Query: When did President Trump and Vice President Vance meet with Ukrainian President Volodymyr Zelenskyy in the Oval Office?

🤖 Running query through agent...

➡️ Final answer: Based on the information retrieved, I can answer your question about when President Trump and Vice President Vance met with Ukrainian President Volodymyr Zelenskyy in the Oval Office.

According to the White House website, President Trump hosted President Volodymyr Zelenskyy of Ukraine in the Oval Office on February 28, 2025. This information comes from a video title on the White House website: "President Trump and Ukrainian President Zelenskyy in Oval Office, Feb. 28, 2025".

However, I must note that the retrieved information does not specifically mention Vice President Vance being present at this meeting. The sources only refer to President Trump and President Zelenskyy. If Vice President Vance was indeed part of this meeting, that information is not provided in the context I have.

To summarize:
- The m

## Review detections in Quotient

You can now view your logs and detections in the [Quotient dashboard](app.quotientai.co), where you can also filter them by tags and environments to identify common failure patterns.

![Quotient AI Dashboard](Agent_Monitoring.png "Quotient AI Dashboard")

### What You've Built

You've built a fully functional, monitoring-included research agent that:

- Handles real-time web queries using Exa + OpenAI
- Retrieves and extracts live documents with state-of-the-art search capabilities
- Automatically logs each response with Quotient for monitoring
- Flags hallucinations, irrelevant context, or broken reasoning
- Leverages Exa's Python SDK for seamless integration

This setup can scale from notebook experiments to production pipelines, letting you benchmark different models, debug search performance, and monitor AI agents for critical issues.

> ### How to interpret the results
> - Well-grounded systems typically show **< 5% hallucination rate**. If yours is higher, it’s often a signal that either your data ingestion, retrieval pipeline, or prompting needs improvement.
> - High-performing systems typically show **> 75% document relevance**. Lower scores may signal ambiguous user queries, incorrect retrieval, or noisy source data.

## Optional: Grab the detection results from Quotient

Quotient's detections are now available to fetch via the Quotient SDK using the `log_id` you received earlier:

In [6]:
from tqdm import tqdm

hallucination_detections = []
doc_relevancy_detections = []

for id in tqdm(log_ids):
    try:
        detection = quotient.poll_for_detection(log_id=id)
        # Add the hallucination detection to the hallucination_detections list
        hallucination_detections.append(detection.has_hallucination)
        # Add the document relevancy detection to the doc_relevancy_detections list
        docs = detection.log_documents
        doc_relevancy_detections.append(sum(1 for doc in docs if doc.get('is_relevant') is True) / len(docs) if docs else None)
    except:
        continue

print(f"Number of results: {len(log_ids)}")
print(f"Percentage of hallucinations: {sum(hallucination_detections)/len(hallucination_detections)*100:.2f}%")
print(f"Average percentage of relevant documents: {sum(doc_relevancy_detections)/len(doc_relevancy_detections)*100:.2f}%")

100%|██████████| 10/10 [00:43<00:00,  4.30s/it]

Number of results: 10
Percentage of hallucinations: 90.00%
Average percentage of relevant documents: 28.33%
